# Meta Agent
By: Linxin Song, Jiale Liu, Jieyu Zhang

In this notebook, we introduce MetaAgent, an agent enhanced with the capability to call AutoBuild to break down and solve complex tasks. AutoBuild can initiate a nested chat between a group of experts and converse to solve the task. The experts in nested chat can be retrieved from agent library. The agents can be equipped with tools for advanced coding. In addition to agent library, we release a tool library that can assist with advanced tasks.


## Setup API endpoint
In order to setup API, you should create a OAI_CONFIG_LIST file. The config list should look like the following:
```python
config_list = [
    {
        'model': 'gpt-4-1106-preview',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    },
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    },
]
```

In [1]:
import autogen

config_file_or_env = "OAI_CONFIG_LIST"
llm_config = {"temperature": 0}
config_list = autogen.config_list_from_json(
    config_file_or_env, filter_dict={"model": ["gpt-4-1106-preview", "gpt-4"]}
)  # You can modify the filter_dict to select the models

## Building Agents from library & Retrieve tools from tool library
One significant feature of MetaAgent is that the experts and tools can be retrieved from a dedicated library. `metaagent_expert_library.json` contains examples of experts. You can add to it following the format. The tools are stored in `tools` directory, it is categorized into three types: data analysis, information_retrieval, math. In order for the functions to work properly, refer to the readme for further details.

In [2]:
from autogen.agentchat.contrib.meta_agent import MetaAgent
from autogen.agentchat.contrib.meta_user_proxy_agent import MetaUserProxyAgent

general_llm_config = {
    "temperature": 0,
    "config_list": autogen.config_list_from_json("OAI_CONFIG_LIST", filter_dict={"model": ["gpt-4-1106-preview"]}),
}

nested_mode_config = {
    "autobuild_init_config": {
        "config_file_or_env": "OAI_CONFIG_LIST",
        "builder_model": "gpt-4-1106-preview",
        "agent_model": "gpt-4-1106-preview",
    },
    # this is used to configure the autobuild building process
    "autobuild_build_config": {
        "default_llm_config": {"temperature": 1, "top_p": 0.95, "max_tokens": 1500, "seed": 52},
        # this is used to configure the user proxy within nested chat
        "code_execution_config": {"timeout": 300, "work_dir": "groupchat", "last_n_messages": 1},
        "coding": True,
        "library_path_or_json": "metaagent_expert_library.json",
    },
    "autobuild_tool_config": {
        "tool_corpus": "../tools/tool_description.tsv",
        "tool_root": "../tools",
        "retriever": "all-mpnet-base-v2",
    },
    "group_chat_config": {"max_round": 15},
    "group_chat_llm_config": general_llm_config.copy(),
}

/home/vscode/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now let's build the agents and ask it to solve a problem.

In [3]:
## build agents
meta_agent = MetaAgent(name="meta_agent", llm_config=general_llm_config, nested_mode="autobuild")
meta_user_proxy = MetaUserProxyAgent(
    name="meta_user_proxy",
    nested_mode_config=nested_mode_config,
    code_execution_config={"use_docker": False},  # you can modify the setting
    agent_config_save_path=None,  # If you'd like to save the created agents in nested chat for further use, specify the path here
)

In [4]:
# The function requires BING api key and Rapid API key to work. You can follow the instructions from readme to get one.
import os

os.environ["BING_API_key"] = ""
os.environ["RAPID_API_KEY"] = ""

query = """Today's date is 2024-04-14.
# Task
You need to solve the below question given by a user.

# Question
Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec.

What does Teal'c say in response to the question "Isn't that hot?"
""".strip()
meta_user_proxy.initiate_chat(meta_agent, message=query)

meta_user_proxy (to meta_agent):

Today's date is 2024-04-14.
# Task
You need to solve the below question given by a user.

# Question
Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec.

What does Teal'c say in response to the question "Isn't that hot?"

--------------------------------------------------------------------------------
meta_agent (to meta_user_proxy):

To solve this task, I will perform the following steps:

1. Identify a reliable way to extract video captions or subtitles from the YouTube video. As of my last knowledge update in April 2023, YouTube videos often contain automatic or user-provided closed captions that can be accessed using YouTube's API or third-party tools.
2. Analyze the captions or subtitles around the timestamp where the question "Isn't that hot?" is asked to find Teal'c's response.
3. Verify that the retrieved response matches the context of the question posed by reviewing the adjacent dialogues.

Given these steps, I will need a group

ChatResult(chat_id=None, chat_history=[{'content': 'Today\'s date is 2024-04-14.\n# Task\nYou need to solve the below question given by a user.\n\n# Question\nExamine the video at https://www.youtube.com/watch?v=1htKBjuUWec.\n\nWhat does Teal\'c say in response to the question "Isn\'t that hot?"', 'role': 'assistant'}, {'content': 'To solve this task, I will perform the following steps:\n\n1. Identify a reliable way to extract video captions or subtitles from the YouTube video. As of my last knowledge update in April 2023, YouTube videos often contain automatic or user-provided closed captions that can be accessed using YouTube\'s API or third-party tools.\n2. Analyze the captions or subtitles around the timestamp where the question "Isn\'t that hot?" is asked to find Teal\'c\'s response.\n3. Verify that the retrieved response matches the context of the question posed by reviewing the adjacent dialogues.\n\nGiven these steps, I will need a group of experts with the following skills:\n\

The ground truth answer to the question is 'Extremely', which the MetaAgent answers correctly. Notably, with the assistance of tools, the agent can answer video-related questions. This shows the huge potential of tools. 